In [1]:
from ler.rates import LeR
from ler.utils import get_param_from_json
import numpy as np
import matplotlib.pyplot as plt

# other necessary imports
from astropy.cosmology import LambdaCDM

Setting multiprocessing start method to 'fork'


In [4]:
# for checking all available model functions
ler = LeR(verbose=False);

In [9]:
# for checking all available model functions
print(f'Available optical functions: \n{ler.available_optical_depth_list_and_its_params.keys()}')
print(f'Available velocity dispersion functions: \n{ler.available_velocity_dispersion_list_and_its_params.keys()}')

Available optical functions: 
dict_keys(['optical_depth_SIS_haris', 'optical_depth_SIS_hemanta', 'optical_depth_SIE_hemanta'])
Available velocity dispersion functions: 
dict_keys(['velocity_dispersion_haris', 'velocity_dispersion_gengamma', 'velocity_dispersion_bernardi', 'velocity_dispersion_ewoud'])


In [36]:
# For SIS
def sample_axis_ratio(sigma):
    return np.zeros_like(sigma)

def sample_axis_rotation_angle(size):
    return np.zeros(size)

def sample_shear(size):
    return np.zeros((2,size))

def sample_density_profile_slope(size):
    return np.ones(size)*(2.0)

In [37]:
# Complete setup for LeR

ler = LeR(
    # LeR setup arguments
    npool=8, # number of processors to use
    z_min=0.0, # minimum redshift
    z_max=10.0, # maximum redshift
    event_type='BBH', # event type
    size=100000, # number of events to simulate
    batch_size=50000, # batch size
    cosmology=LambdaCDM(H0=70, Om0=0.3, Ode0=0.7), # cosmology
    snr_finder=None, # snr calculator from 'gwsnr' package will be used
    pdet_finder=None,  # will not be consider unless specified
    list_of_detectors=None, # list of detectors that will be considered when calculating snr or pdet for lensed events. if None, all the detectors from 'gwsnr' will be considered
    json_file_names=dict(
        ler_params="ler_params.json", # to store initialization parameters and important results
        unlensed_param="unlensed_param.json", # to store all unlensed events
        unlensed_param_detectable="unlensed_param_detectable.json", # to store only detectable unlensed events
        lensed_param="lensed_param.json", # to store all lensed events 
        lensed_param_detectable="lensed_param_detectable.json"), # to store only detectable lensed events
    interpolator_directory='./interpolator_pickle', # directory to store the interpolator pickle files. 'ler' uses interpolation to get values of various functions to speed up the calculations (relying on numba njit).
    create_new_interpolator = False, # if True, will create new interpolator files
    ler_directory='./ler_data', # directory to store all the outputs
    verbose=False, # if True, will print all information at initialization

    # CBCSourceParameterDistribution class arguments
    source_priors= {
        'merger_rate_density': 'merger_rate_density_bbh_popI_II_oguri2018', 'source_frame_masses': 'binary_masses_BBH_popI_II_powerlaw_gaussian', 'zs': 'sample_source_redshift', 'geocent_time': 'sampler_uniform', 'ra': 'sampler_uniform', 'dec': 'sampler_cosine', 'phase': 'sampler_uniform', 'psi': 'sampler_uniform', 'theta_jn': 'sampler_sine'
        },
    source_priors_params= {
        'merger_rate_density': {
            'R0': 2.39e-08, 'b2': 1.6, 'b3': 2.0, 'b4': 30
            }, 
        'source_frame_masses': {
            'mminbh': 4.98, 'mmaxbh': 112.5, 'alpha': 3.78, 'mu_g': 32.27, 'sigma_g': 3.88, 'lambda_peak': 0.03, 'delta_m': 4.8, 'beta': 0.81
            }, 
        'zs': None, 
        'geocent_time': {
            'min_': 1238166018, 'max_': 1269702018
            }, 
        'ra': {
            'min_': 0.0, 'max_': 6.283185307179586
            }, 
        'dec': None, 
        'phase': {
            'min_': 0.0, 'max_': 6.283185307179586
            }, 
        'psi': {
            'min_': 0.0, 'max_': 3.141592653589793
            }, 
        'theta_jn': None
        },
    spin_zero= True, # if True, spins will be set to zero
    spin_precession= False, # if True, spins will be precessing

    # LensGalaxyParameterDistribution class arguments
    lens_type = 'epl_shear_galaxy',
    lens_functions =  {
        'strong_lensing_condition': 'rjs_with_cross_section_SIS',
        'optical_depth': 'optical_depth_SIS_haris',  # SIS model is used for optical depth calculation
        'param_sampler_type': 'sample_all_routine'
        },
    lens_priors =  {
        'source_redshift_sl': 'strongly_lensed_source_redshifts', 
        'lens_redshift': 'lens_redshift_SDSS_catalogue', 
        'velocity_dispersion': 'velocity_dispersion_gengamma', # Haris et al. 2018
        'axis_ratio': sample_axis_ratio,  # for SIS 
        'axis_rotation_angle': sample_axis_rotation_angle,  # for SIS  
        'external_shear': sample_shear,  # for SIS
        'density_profile_slope': sample_density_profile_slope,  # for SIS
        'source_parameters': 'sample_gw_parameters'
        },
    lens_priors_params =  {
        'source_redshift_sl': None, 
        'lens_redshift': None, 
        'velocity_dispersion': {
            'vd_min':10., 'vd_max':350.
            }, 
        'axis_ratio': None, 
        'axis_rotation_angle': None, 
        'external_shear': None, 
        'density_profile_slope': None,
        },

    # ImageProperties class arguments
    n_min_images = 2,
    n_max_images = 4,
    geocent_time_min = 1238166018,
    geocent_time_max = 1269702018,
    # lens_model_list = ['EPL_NUMBA', 'SHEAR'],  

    # gwsnr package arguments
    mtot_min = 2.0,
    mtot_max = 184.98599853446768,
    ratio_min = 0.1,
    ratio_max = 1.0,
    # mtot_resolution = 500,
    # ratio_resolution = 50,
    sampling_frequency = 2048.0,
    waveform_approximant = 'IMRPhenomXPHM',
    minimum_frequency = 20.0,
    snr_type = 'inner_product',
    psds = {'L1':'aLIGO_O4_high_asd.txt','H1':'aLIGO_O4_high_asd.txt', 'V1':'AdV_asd.txt'},
    ifos = ['L1', 'H1', 'V1'],
    interpolator_dir = './interpolator_pickle',
    # gwsnr_verbose = True,
    # multiprocessing_verbose = True,
    mtot_cut = True,
)

In [34]:
ler.selecting_n_unlensed_detectable_events(
    size=1000,
    batch_size=25000,
    snr_threshold=10.0,
    resume=True,
    output_jsonfile='n_unlensed_param_detectable_O4_Jose.json',
    meta_data_file='meta_unlensed_O4_Jose.json',
    detectability_condition='step_function',
    trim_to_size=False,
);

  0%|                                                                     | 0/23135 [00:00<?, ?it/s]

collected number of detectable events =  573


  0%|                                                                     | 0/23118 [00:00<?, ?it/s]

given detectability_condition == 'step_function'
collected number of detectable events =  693
total number of events =  150000
total rate (yr^-1): 478.2757823151215


  0%|                                                                     | 0/23092 [00:00<?, ?it/s]

given detectability_condition == 'step_function'
collected number of detectable events =  811
total number of events =  175000
total rate (yr^-1): 479.7546808380501


  0%|                                                                     | 0/23197 [00:00<?, ?it/s]

given detectability_condition == 'step_function'
collected number of detectable events =  923
total number of events =  200000
total rate (yr^-1): 477.75816783209643


100%|████████████████████████████████████████████████████████| 23197/23197 [00:26<00:00, 884.85it/s]


given detectability_condition == 'step_function'
collected number of detectable events =  1037
total number of events =  225000
total rate (yr^-1): 477.1255279083992
stored detectable unlensed params in ./ler_data/n_unlensed_param_detectable_O4_Jose.json
stored meta data in ./ler_data/meta_unlensed_O4_Jose.json


In [39]:
ler.selecting_n_lensed_detectable_events(
    size=1000,
    batch_size=25000,
    snr_threshold=[8.0, 8.0],
    num_img=[1, 1],
    resume=True,
    detectability_condition='step_function',
    output_jsonfile='n_lensed_param_detectable_O4_Jose.json',
    meta_data_file='meta_lensed_O4_Jose.json',
    trim_to_size=False,
    nan_to_num=False,
);

collected number of detectable events =  582


100%|██████████████████████████████████████████████████████████████| 57/57 [00:00<00:00, 248.46it/s]


given detectability_condition == step_function
collected number of detectable events =  637
total number of events =  350000
total rate (yr^-1): 0.1839623544246117


100%|██████████████████████████████████████████████████████████████| 59/59 [00:00<00:00, 256.41it/s]


given detectability_condition == step_function
collected number of detectable events =  693
total number of events =  375000
total rate (yr^-1): 0.18679254449268265


...

given detectability_condition == step_function
collected number of detectable events =  736
total number of events =  400000
total rate (yr^-1): 0.1859839187589481


100%|██████████████████████████████████████████████████████████████| 53/53 [00:00<00:00, 219.48it/s]


given detectability_condition == step_function
collected number of detectable events =  770
total number of events =  425000
total rate (yr^-1): 0.1831299455810614


 76%|██████████████████████████████████████████▊             | 19114/25000 [00:31<00:09, 613.93it/s]


KeyboardInterrupt: 